# Gen AI Demo

__Import OpenAI key__

In [ ]:
from secret_key import openai_api_key

__Import OpenAI, ChatOpenAI, PromptTemplate, ChatPromptTemplate__

In [ ]:
%pip install -qU langchain-openai
%pip install -qU langchain-core
%pip install -qU langchain
%pip install -qU langchain-community

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

__Use LangChain OpenAI, LangChain ChatOpenAI__

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key=openai_api_key, temperature=0.7)
llm.invoke("How far is the moon away from earth in miles ?")

In [ ]:
chat_llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.7)
chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You are an expert in Python programming language"),
        HumanMessagePromptTemplate.from_template("Tell me about scikit learn in 2 sentences")
])
                                     
chain = chat_prompt_template | chat_llm

chain.invoke({})

__ChatPromptTemplate with dynamic values__

In [ ]:
chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You are an expert in {main_topic}"),
        HumanMessagePromptTemplate.from_template("Tell me about {sub_topic} in 2 sentences")
])
                                     
chain = chat_prompt_template | chat_llm

chain.invoke({"main_topic": "Node.js", "sub_topic": "closures"})

In [ ]:
chain.invoke({"main_topic": "SAP Integration Suite", "sub_topic": "OData Adapter"})

__Output Parsers - Format the output for APIs__

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({"main_topic": "Node.js", "sub_topic": "closures"})

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You have extensive knowledge about all the countries in the world"),
        HumanMessagePromptTemplate.from_template("Name 5 big cities in the following country: {country}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({"country": "Sweden"})

# print(type(chain.invoke({"country": "Sweden"})))

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Employee(BaseModel):
    name: str = Field(description="Name of person")
    company: str = Field(description="Name of company")
    experience: int = Field(description="Number of years with company")

output_parser = JsonOutputParser(pydantic_object=Employee)

chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("Create a fictitious employee with the following name.  \nFormatting Instructions: {format_instructions}"),
        HumanMessagePromptTemplate.from_template("Employee name: {name}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({
    "name": "Milton",
    "format_instructions": output_parser.get_format_instructions()
    })

# print(type(chain.invoke({
#     "name": "Milton",
#     "format_instructions": output_parser.get_format_instructions()
#     })
# ))

__Summarization Techniques: Stuff Chain, Map Reduce, Refine__

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_retrieval_chain

from langchain.document_loaders import TextLoader
my_character = TextLoader('my_character.txt').load()

In [ ]:
summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="stuff")
summary = summarize_chain.invoke(my_character)

print(summary["output_text"])

In [ ]:
map_template_string = """
Write a concise summary of the following text delimited by 3 backticks.

text: ```{text}```
       
SUMMARY:
"""

map_prompt_template = PromptTemplate(template=map_template_string, input_variables=["text"])


combine_template_string = """
Provide a final concise summary of the following text delimited by 3 backticks.
       
text: ```{text}```
"""
combine_prompt_template = PromptTemplate(template=combine_template_string, input_variables=['text'])

summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="map_reduce", map_prompt=map_prompt_template, combine_prompt=combine_prompt_template)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

documents = text_splitter.split_documents(my_character)

# print(type(documents[0]))

summary = summarize_chain.invoke(documents)
print(summary["output_text"])


In [ ]:
summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="refine")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

documents = text_splitter.split_documents(my_character)

# print(type(documents[0]))

summary = summarize_chain.invoke(documents)
print(summary["output_text"])
